# Applying Spell Correction for Text preprocessing

This is an ongoing improvement on the core implemenation of the Spell <br>checking algorithm created By Peter Norvig
for more information on [modeling basicss](http://norvig.com/spell-correct.html)

In [1]:
from nltk.metrics import edit_distance

In [17]:
import os
os.listdir()

['.ipynb_checkpoints',
 'Basic_Processing.ipynb',
 'data',
 'HTML - Text Extraction.ipynb',
 'Loading_from_Filesources.ipynb',
 'Spell_correction.ipynb',
 'Stemming_Lemmatizing_Words.ipynb',
 'Untitled.ipynb',
 'Working_with_sentences.ipynb']

In [66]:
# Implementing a basic spell checker from Peter Norvig
import re
import nltk
from collections import Counter

# Tokenize the words
def words(text): return re.findall(r'\w+', text.lower())
WORDS = Counter(words(open('./data/big.txt', 'r').read()))

In [109]:
def check_spelling(text):
    """
    Spell corrects a text for the English language. 
    
    Given a string of Text and a trained FreqDict of a given language, the model
    tries to Spell correct the sentences keeping original Formatting and Casing in place.
    """
    

    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    
    def splits(word):
        "Return a list of all possible (first, rest) pairs that comprise word."
        return [(word[:i], word[i:]) for i in range(len(word)+1)]
    
    def correct(word):
        "Find the best spelling correction for this word."
        # Prefer edit distance 0, then 1, then 2; otherwise default to word itself.
        candidates = (known(edits0(word)) or 
                      known(edits1(word)) or 
                      known(edits2(word)) or 
                      [word])
        return max(candidates, key=WORDS.get)
    
    def correct_text(text):
        "Correct all the words within a text, returning the corrected text."
        return re.sub('[a-zA-Z]+', correct_match, text)
    
    def correct_match(match):
        "Spell-correct word in match, and preserve proper upper/lower/title case."
        word = match.group()
        return case_of(word)(correct(word.lower()))
    
    def case_of(text):
        "Return the case-function appropriate for text: upper, lower, title, or just str."
        return (str.upper if text.isupper() else
                str.lower if text.islower() else
                str.title if text.istitle() else
                str)
    
    def candidates(word):
        "Generate probable spelling corrections for word"
        return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])
    
    def known(words):
        "The subset of 'words' that appear in the dictionary of WORDS"
        return {w for w in words if w in WORDS}
    
    def edits0(word):
        return {word}
    
    def edits1(word):
        "Return all strings that are one edit away from this word."
        pairs      = splits(word)
        deletes    = [a+b[1:]           for (a, b) in pairs if b]
        transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
        replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
        inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
        return set(deletes + transposes + replaces + inserts)

    def edits2(word):
        "All edits that are two edits away from word"
        return {e2 for e1 in edits1(word) for e2 in edits1(e1)}
    
    return correct_text(text)

In [110]:
check_spelling('Speling')

'Spelling'

In [111]:
WORDS.most_common(10)

[('the', 79809),
 ('of', 40024),
 ('and', 38312),
 ('to', 28765),
 ('in', 22023),
 ('a', 21124),
 ('that', 12512),
 ('he', 12401),
 ('was', 11410),
 ('it', 10681)]

In [112]:
test = "Speling Errurs IN somethink. Whutever; unusuel misteakes?"
check_spelling(test)

'Spelling Errors IN something. Whatever; unusual mistakes?'

### Calculating the joint propability of a given sentence

We should be able to compute the probability of a word, P(w)P(w). We do that with the function pdist, which takes as input a Counter (hat is, a bag of words) and returns a function that acts as a probability distribution over all possible words. In a probability distribution the probability of each word is between 0 and 1, and the sum of the probabilities is 1.

In [96]:
def pdist(counter):
    "Calculates the prior propability of a given Word"
    N = sum(counter.values())
    return lambda x: counter[x]/N

def product(nums):
    "Multiply the numbers together, like sum but with multiply"
    result = 1
    for x in nums:
        result *= x
    return result

Pword = pdist(WORDS)

def Pwords(words):
    "Propability of word"
    return product(Pword(w) for w in words)

In [104]:
tests = ['this is a test', 
         'this is a unusual test',
         'this is a neverbeforeseen test']

[Pwords(words(text)) for text in tests]

[2.870225938180123e-11, 8.233100124308226e-16, 0.0]

### Using sentence propability count to solve Word Segmentation

Make one segmentation, into a first word and remaining characters. If we assume words are independent then we can maximize the probability of the first word adjoined to the best segmentation of the remaining characters.

assert segment('choosespain') == ['choose', 'spain']

segment('choosespain') ==
   max(Pwords(['c'] + segment('hoosespain')),<br>
       Pwords(['ch'] + segment('oosespain')),<br>
       Pwords(['cho'] + segment('osespain')),<br>
       Pwords(['choo'] + segment('sespain')),<br>
       ...<br>
       Pwords(['choosespain'] + segment('')))<br>
       
<br><br>
To make this somewhat efficient, we need to avoid re-computing the segmentations of the remaining characters. This can be done explicitly by dynamic programming or implicitly with memoization. Also, we shouldn't consider all possible lengths for the first word; we can impose a maximum length. What should it be? A little more than the longest word seen so far.


In [118]:
def memo(f):
    "Memoize function f, whose args must all be hashable"
    cache = {}
    def fmemo(*args):
        if args not in cache:
            cache[args] = f(*args)
        return cache[args]
    fmemo.cache = cache
    return fmemo

In [106]:
max(len(w) for w in WORDS)

18

In [107]:
def splits(text, start=0, L=20):
    "Returns a list of all (first, rest) pairs: start <= len(first) <= L"
    return [(text[:i], text[i:])  for i in range(start, min(len(text), L)+1)]

In [115]:
splits('reallylongtextwithacoupleofwords', 1,10)

[('r', 'eallylongtextwithacoupleofwords'),
 ('re', 'allylongtextwithacoupleofwords'),
 ('rea', 'llylongtextwithacoupleofwords'),
 ('real', 'lylongtextwithacoupleofwords'),
 ('reall', 'ylongtextwithacoupleofwords'),
 ('really', 'longtextwithacoupleofwords'),
 ('reallyl', 'ongtextwithacoupleofwords'),
 ('reallylo', 'ngtextwithacoupleofwords'),
 ('reallylon', 'gtextwithacoupleofwords'),
 ('reallylong', 'textwithacoupleofwords')]

In [120]:
@memo
def segment(text):
    "Return a list of words that is the most probable segmentation of text"
    if not text:
        return []
    else:
        candidates = ([first] + segment(rest) for (first, rest) in splits(text,1))
        return max(candidates, key=Pwords)

In [122]:
segment('reallylongtextwithacoupleofwords')

['really', 'long', 'text', 'with', 'a', 'couple', 'of', 'words']